In [1]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import shapely

import osm_flex.download as dl
import osm_flex.extract as ex
from osm_flex.config import OSM_DATA_DIR

from tqdm import tqdm

from lonboard import viz
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Blues_9

from pathlib import Path
import pathlib

In [2]:
# define paths
p = Path("..")
data_path = Path(pathlib.Path.home().parts[0]) / "Data"
flood_data = data_path / "Flood_data"
vul_data = data_path / "Vulnerability"

### LOAD OSM DATA

In [95]:
iso3 = "JAM"
dl.get_country_geofabrik(iso3)

data_loc = OSM_DATA_DIR.joinpath("jamaica-latest.osm.pbf")

assets = ex.extract_cis(data_loc, "road")
assets = assets.loc[assets.geometry.geom_type == "LineString"]
assets = assets.rename(columns={"highway": "asset"})

extract points: 0it [00:00, ?it/s]
extract lines: 100%|███████████████████████████████████████████████████████████| 39952/39952 [00:04<00:00, 9838.46it/s]


In [96]:
viz(assets, width_min_pixels=1)

Map(layers=[PathLayer(table=pyarrow.Table
osm_id: string
asset: string
name: string
maxspeed: string
lanes: st…

### LOAD HAZARD DATA

In [5]:
pluvial_data = flood_data / "Jamaica" / "fluvial_undefended"

In [6]:
fluvial_flood = list(pluvial_data.iterdir())[2]

In [8]:
flood_map = xr.open_dataset(fluvial_flood, engine="rasterio")

In [9]:
flood_map_vector = flood_map["band_data"].to_dataframe().reset_index()

# remove data that will not be used
flood_map_vector = flood_map_vector.loc[
    (flood_map_vector.band_data > 0) & (flood_map_vector.band_data < 100)
]

# create geometry values and drop lat lon columns
flood_map_vector["geometry"] = [
    shapely.points(x) for x in list(zip(flood_map_vector["x"], flood_map_vector["y"]))
]
flood_map_vector = flood_map_vector.drop(["x", "y", "band", "spatial_ref"], axis=1)

# drop all non values to reduce size
flood_map_vector = flood_map_vector.loc[
    ~flood_map_vector["band_data"].isna()
].reset_index(drop=True)

# and turn them into squares again:
flood_map_vector.geometry = shapely.buffer(
    flood_map_vector.geometry, distance=0.00083 / 2, cap_style="square"
).values

In [10]:
viz(
    gpd.GeoDataFrame(flood_map_vector.copy()),
    get_fill_color=apply_continuous_cmap(flood_map_vector.band_data, Blues_9),
)

Map(layers=[SolidPolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x0000020D08FC6F20>
[
 …

### LOAD VULNERABILITY DATA

In [11]:
curves = pd.read_excel(
    vul_data / "Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx",
    sheet_name="F_Vuln_Depth",
    index_col=[0],
    header=[0, 1, 2, 3, 4],
)
infra_curves = curves.loc[
    :,
    curves.columns.get_level_values("Infrastructure description").str.contains("Roads"),
]

In [12]:
maxdam = pd.read_excel(
    vul_data / "Table_D3_Costs_V1.0.0.xlsx", sheet_name="Cost_Database", index_col=[0]
)

### PERFORM DAMAGE ASSESSMENT

In [13]:
def overlay_hazard_assets(df_ds, assets):
    # overlay
    hazard_tree = shapely.STRtree(df_ds.geometry.values)
    if (shapely.get_type_id(assets.iloc[0].geometry) == 3) | (
        shapely.get_type_id(assets.iloc[0].geometry) == 6
    ):
        return hazard_tree.query(assets.geometry, predicate="intersects")
    else:
        return hazard_tree.query(assets.buffered, predicate="intersects")


def buffer_assets(assets, buffer_size=0.00083):
    assets["buffered"] = shapely.buffer(assets.geometry.values, distance=buffer_size)
    return assets


def get_damage_per_asset(asset, flood_numpified, asset_geom, asset_type, curve, maxdam):
    # find the exact hazard overlays:
    get_hazard_points = flood_numpified[asset[1]["hazard_point"].values]
    get_hazard_points[shapely.intersects(get_hazard_points[:, 1], asset_geom)]

    # get maxdam and curves
    maxdam_asset = maxdam
    hazard_intensity = curve.index.values
    fragility_values = curve.iloc[:, 0].values

    # estimate damage
    if len(get_hazard_points) == 0:
        return asset[0], 0
    else:
        overlay_meters = shapely.length(
            shapely.intersection(get_hazard_points[:, 1], asset_geom)
        )
        return asset[0], np.sum(
            (
                np.interp(
                    np.float16(get_hazard_points[:, 0]),
                    hazard_intensity,
                    fragility_values,
                )
            )
            * overlay_meters
            * maxdam_asset
        )

In [14]:
%%time
overlay_roads = pd.DataFrame(
    overlay_hazard_assets(flood_map_vector, buffer_assets(roads)).T,
    columns=["asset", "hazard_point"],
)

CPU times: total: 4.3 s
Wall time: 4.3 s


In [15]:
maxdams = maxdam.loc["Roads", "Amount"].dropna()[:10]

In [90]:
collect_output = {}
geom_dict = roads["geometry"].to_dict()
type_dict = roads["asset"].to_dict()

flood_numpified = flood_map_vector.to_numpy()
for infra_curve in infra_curves:
    curve = infra_curves[infra_curve[0]]
    for maxdam in maxdams:
        for asset in tqdm(
            overlay_roads.groupby("asset"),
            total=len(overlay_roads.asset.unique()),
            desc=f"polyline damage calculation for {infra_curve[0]} with a max dam of {maxdam}",
        ):
            asset_geom = geom_dict[asset[0]]
            asset_type = type_dict[asset[0]]
            collect_output[infra_curve[0], maxdam] = get_damage_per_asset(
                asset, flood_numpified, asset_geom, asset_type, curve, maxdam
            )

polyline damage calculation for F7.1 with a max dam of 909.3454827565133: 100%|██| 5994/5994 [00:02<00:00, 2729.71it/s]
polyline damage calculation for F7.1 with a max dam of 909.3454827565133: 100%|██| 5994/5994 [00:02<00:00, 2752.55it/s]
polyline damage calculation for F7.1 with a max dam of 7.7971745573977564: 100%|█| 5994/5994 [00:02<00:00, 2758.52it/s]
polyline damage calculation for F7.1 with a max dam of 5746.447358140652: 100%|██| 5994/5994 [00:02<00:00, 2678.17it/s]
polyline damage calculation for F7.1 with a max dam of 1340.8377168994855: 100%|█| 5994/5994 [00:02<00:00, 2659.77it/s]
polyline damage calculation for F7.1 with a max dam of 660.841446186175: 100%|███| 5994/5994 [00:02<00:00, 2561.29it/s]
polyline damage calculation for F7.1 with a max dam of 95.77412263567753: 100%|██| 5994/5994 [00:02<00:00, 2665.75it/s]
polyline damage calculation for F7.1 with a max dam of 114.92894716281305: 100%|█| 5994/5994 [00:02<00:00, 2647.03it/s]
polyline damage calculation for F7.1 wit